## Maps

In [1]:
import pandas as pd
import geopandas
import os

In [2]:
dir_maps = os.path.join('output', 'sparky11111111111', 'combined', 'maps')

In [3]:
# Load in the data

gdf_boundaries_regions = pd.read_csv(
    os.path.join(dir_maps, 'gdf_boundaries_regions.csv'),
    index_col=[0],
    header=[0, 1],
)
gdf_points_units = pd.read_csv(
    os.path.join(dir_maps, 'gdf_points_units.csv'),
    index_col=0,
    header=[0, 1],
)
gdf_lines_transfer = pd.read_csv(
    os.path.join(dir_maps, 'gdf_lines_transfer.csv'),
    index_col=[0],
    header=[0, 1],
)
gdf_boundaries_lsoa = pd.read_csv(
    os.path.join(dir_maps, 'gdf_boundaries_lsoa.csv'),
    index_col=[0],
    header=[0, 1, 2],
)
gdf_boundaries_catchment = pd.read_csv(
    os.path.join(dir_maps, 'gdf_boundaries_catchment.csv'),
    index_col=[0],
    header=[0, 1],
)

In [4]:
from shapely import wkt

def convert_df_to_gdf(df):
    df = df.copy()
    # Find the intended geometry column in the "property" column level:
    col = df.columns[df.columns.get_level_values('property').isin(['geometry'])].values[0]
    try:
        gdf = geopandas.GeoDataFrame(
            df,
            geometry=col,
            # crs=gdfs_to_merge[0].crs
            )
    except TypeError:
        # # Convert to a proper geometry column:
        df[col] = df[col].apply(wkt.loads)
        # Convert to a proper GeoDataFrame:
        gdf = geopandas.GeoDataFrame(
            df,
            geometry=col,
            # crs=gdfs_to_merge[0].crs
            )
    return gdf

In [5]:
gdf_boundaries_regions = convert_df_to_gdf(gdf_boundaries_regions)
gdf_points_units = convert_df_to_gdf(gdf_points_units)
gdf_lines_transfer = convert_df_to_gdf(gdf_lines_transfer)
gdf_boundaries_lsoa = convert_df_to_gdf(gdf_boundaries_lsoa)
gdf_boundaries_catchment = convert_df_to_gdf(gdf_boundaries_catchment)

In [6]:
gdf_points_units.head(3)

property           postcode                                stroke_team  \
scenario Unnamed: 1_level_1                                        any   
id                                                                       
0                    B152TH       Queen Elizabeth Hospital, Birmingham   
1                    B714HJ  Sandwell District Hospital, West Bromwich   
2                     B95SS             Birmingham Heartlands Hospital   

property short_code                          ssnap_name  lsoa_code  \
scenario        any                                 any        any   
id                                                                   
0                QE  Queen Elizabeth Hospital Edgbaston  E01033562   
1                SW          Sandwell District Hospital  E01010104   
2                BM      Birmingham Heartlands Hospital  E01009341   

property                                 region region_code region_type  \
scenario                                    any         any         any   
id                                                                        
0         NHS Birmingham and Solihull ICB - 15E   E38000258       SICBL   
1                 NHS Black Country ICB - D2P2L   E38000259       SICBL   
2         NHS Birmingham and Solihull ICB - 15E   E38000258       SICBL   

property  country                                                icb  ...  \
scenario      any                                                any  ...   
id                                                                    ...   
0         England  NHS Birmingham and Solihull Integrated Care Board  ...   
1         England            NHS Black Country Integrated Care Board  ...   
2         England  NHS Birmingham and Solihull Integrated Care Board  ...   

property    use_msu   selected transfer_unit_postcode periphery_unit  \
scenario mothership mothership             mothership  drip-and-ship   
id                                                                     
0                 1          0                nearest              0   
1                 0          0                nearest              0   
2                 0          0                nearest              0   

property                BNG_E     BNG_N   Latitude Longitude  \
scenario mothership       any       any        any       any   
id                                                             
0                 0  404427.0  283926.0  52.453272 -1.936284   
1                 0  400919.0  292165.0  52.527357 -1.987884   
2                 0  411645.0  286935.0  52.480218 -1.829966   

property                       geometry  
scenario            Unnamed: 29_level_1  
id                                       
0         POINT (404427.000 283926.000)  
1         POINT (400919.000 292165.000)  
2         POINT (411645.000 286935.000)  

[3 rows x 29 columns]

In [7]:
for gdf in [gdf_boundaries_regions, gdf_points_units, gdf_lines_transfer, gdf_boundaries_lsoa, gdf_boundaries_catchment]:
    print(type(gdf))

<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'>


In [8]:
import classes.map as maps

In [9]:
(
gdf_boundaries_regions,
gdf_points_units,
gdf_boundaries_catchment,
gdf_boundaries_lsoa,
gdf_lines_transfer
        ) = maps.crop_data_to_shared_extent(
    gdf_boundaries_regions,
    gdf_points_units,
    gdf_boundaries_catchment,
    gdf_boundaries_lsoa,
    gdf_lines_transfer,
    leeway=20000
    )

<class 'geopandas.geodataframe.GeoDataFrame'> <class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'> <class 'geopandas.geodataframe.GeoDataFrame'>
<class 'geopandas.geodataframe.GeoDataFrame'> <class 'geopandas.geodataframe.GeoDataFrame'>
                                             geometry
0   POLYGON ((422030.096 199080.267, 420824.313 19...
1   POLYGON ((381208.906 186560.391, 382055.402 18...
2   MULTIPOLYGON (((87767.569 8868.285, 89125.548 ...
3   MULTIPOLYGON (((213887.142 143476.485, 213328....
4   POLYGON ((423049.194 232166.407, 423282.814 22...
5   POLYGON ((330421.776 158572.763, 331358.000 15...
6                       POINT (372922.000 166042.000)
7                       POINT (355562.000 116398.000)
8                       POINT (358935.000 177766.000)
9                       POINT (332406.000 158585.000)
10                       POINT (293730.000 91870.000)
11                      POINT (256457.000 134545.000)
12                    

/home/anna/miniconda3/envs/simpy/lib/python3.10/site-packages/shapely/predicates.py:778: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/home/anna/miniconda3/envs/simpy/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/anna/miniconda3/envs/simpy/lib/python3.10/site-packages/geopandas/geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setit

In [11]:
gdf_boundaries_regions

property,region,region_code,BNG_E,BNG_N,GlobalID,LAT,LONG,country,geometry,icb,...,isdn,region_type,short_code,contains_periphery_lsoa,contains_periphery_unit,contains_unit,contains_periphery_lsoa,contains_periphery_unit,contains_unit,point_label
scenario,Unnamed: 1_level_1,Unnamed: 2_level_1,any,any,any,any,any,any,any,any,...,any,any,any,drip-and-ship,drip-and-ship,drip-and-ship,mothership,mothership,mothership,
id,,,,,,,,,,,,,,,,,,,,,
0,Aneurin Bevan University Health Board,W11000028,331785,205411,b6af30e1-0a47-4411-9621-b2037487094e,51.74323,-2.98940,Wales,"MULTIPOLYGON (((345884.597 180997.499, 345877....",NaN,...,NaN,LHB,AB,1,0,0,1,0,0,POINT (331785.5378875477 205486.6544)
1,"NHS Bath and North East Somerset, Swindon and ...",E38000231,393349,154962,e0b99a7f-b497-4c8e-8ca9-fe6a8100eb54,51.29374,-2.09676,England,"POLYGON ((420824.313 197057.500, 421408.497 19...","NHS Bath and North East Somerset, Swindon and ...",...,"Gloucester, BSW, BNSSG and Somerset",SICBL,BA,1,0,1,1,0,0,POINT (406846.61365841026 158184.1950000005)
5,"NHS Buckinghamshire, Oxfordshire and Berkshire...",E38000136,448985,208334,9d6ab1c9-7846-47f9-8863-a09bdd4a79a3,51.77155,-1.29146,England,"POLYGON ((459306.688 215468.109, 459793.000 21...","NHS Buckinghamshire, Oxfordshire and Berkshire...",...,Thames Valley,SICBL,BO1,1,0,0,0,0,0,POINT (440993.88737779006 216411.4396735641)
6,"NHS Buckinghamshire, Oxfordshire and Berkshire...",E38000223,482506,205421,d128f772-891d-4974-a8be-ff75d3652381,51.74143,-0.80644,England,"MULTIPOLYGON (((459306.688 215468.109, 459793....","NHS Buckinghamshire, Oxfordshire and Berkshire...",...,Thames Valley,SICBL,BO2,0,0,0,0,0,0,POINT (459671.422 215692.59917356411)
7,"NHS Buckinghamshire, Oxfordshire and Berkshire...",E38000221,457803,172197,4ce91e18-0545-4f8c-8d67-794e1d6b726c,51.44582,-1.16963,England,"POLYGON ((457408.711 162349.396, 452222.286 16...","NHS Buckinghamshire, Oxfordshire and Berkshire...",...,Thames Valley,SICBL,BO3,1,0,0,0,0,0,POINT (446442.83225 170852.1169999995)
8,"NHS Bristol, North Somerset and South Gloucest...",E38000222,366745,176370,b66b2d21-f477-47bb-a0d1-bf155d02b598,51.48529,-2.48031,England,"POLYGON ((382055.402 185702.105, 381611.906 18...","NHS Bristol, North Somerset and South Gloucest...",...,"Gloucester, BSW, BNSSG and Somerset",SICBL,BR,1,0,1,1,0,1,POINT (364968.5428617727 176925.4213000005)
20,NHS Cornwall and the Isles of Scilly ICB - 11N,E38000089,212497,64493,62c4814f-341b-4912-8a3c-e52520fa4b07,50.45022,-4.64254,England,"MULTIPOLYGON (((89125.548 8184.989, 88583.532 ...",NHS Cornwall and the Isles of Scilly Integrate...,...,"Devon, Cornwall and Isles of Scilly",SICBL,CS,1,0,1,1,0,0,POINT (212732.70400556875 64585.7013999997)
21,Cwm Taf Morgannwg University Health Board,W11000030,296115,196218,7cf0b9ca-6af1-41a1-bbd2-54c8996117bb,51.65512,-3.50313,Wales,"POLYGON ((303262.998 214030.404, 303408.400 21...",NaN,...,NaN,LHB,CT,0,0,0,0,0,0,POINT (301065.7757266753 196001.0995999995)


## Stuff to delete

In [10]:
print(stop, here, please)

NameError: name 'stop' is not defined

In [ ]:
# Load in the data

gdf_boundaries_regions = geopandas.GeoDataFrame(
    gdf_boundaries_regions,
    geometry=gdf_boundaries_regions.columns[gdf_boundaries_regions.columns.get_level_values('property').isin(['geometry'])]
    )
gdf_points_units = geopandas.GeoDataFrame(
    gdf_points_units,
    geometry=gdf_points_units.columns[gdf_points_units.columns.get_level_values('property').isin(['geometry'])]
    )
gdf_lines_transfer = geopandas.GeoDataFrame(
    gdf_lines_transfer,
    geometry=gdf_lines_transfer.columns[gdf_lines_transfer.columns.get_level_values('property').isin(['geometry'])]
    )
gdf_boundaries_lsoa = geopandas.GeoDataFrame(
    gdf_boundaries_lsoa,
    geometry=gdf_boundaries_lsoa.columns[gdf_boundaries_lsoa.columns.get_level_values('property').isin(['geometry'])]
    )
gdf_boundaries_catchment = geopandas.GeoDataFrame(
    gdf_boundaries_catchment,
    geometry=gdf_boundaries_catchment.columns[gdf_boundaries_catchment.columns.get_level_values('property').isin(['geometry'])]
    )

Convert to geopandas

In [ ]:
# Load in the data

gdf_boundaries_regions = pd.read_csv(
    os.path.join(dir_maps, 'gdf_boundaries_regions.csv'),
    index_col=[0, 1],
    header=[0, 1],
)
gdf_points_units = pd.read_csv(
    os.path.join(dir_maps, 'gdf_points_units.csv'),
    index_col=0,
    header=[0, 1],
)
gdf_lines_transfer = pd.read_csv(
    os.path.join(dir_maps, 'gdf_lines_transfer.csv'),
    index_col=[0, 1],
    header=[0, 1],
)
gdf_boundaries_lsoa = pd.read_csv(
    os.path.join(dir_maps, 'gdf_boundaries_lsoa.csv'),
    index_col=[0, 1, 2],
    header=[0, 1],
)
gdf_boundaries_catchment = pd.read_csv(
    os.path.join(dir_maps, 'gdf_boundaries_catchment.csv'),
    index_col=[0],
    header=[0, 1],
)

In [ ]:
gdf_boundaries_regions = geopandas.read_file(
    os.path.join(dir_maps, 'gdf_boundaries_regions.csv'),
    geometry='geometry'
)